In [1]:
!pip install torchmetrics
!pip install comet_ml torch torchvision tqdm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: /usr/bin/python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: /usr/bin/python -m pip install --upgrade pip


In [4]:
import sys
 
sys.path.insert(0, "../")

from EnsembleFramework import Framework

In [17]:
import torch
from torch import nn
class NeuralNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, out_dim, dropout=.0):
        super().__init__()
        self.lin1 = nn.Linear(input_dim , hidden_dim)
        self.lin2 = nn.Linear(hidden_dim, out_dim)

        torch.nn.init.xavier_uniform_(self.lin1.weight) 
        torch.nn.init.xavier_uniform_(self.lin2.weight) 
        self.lin1.bias.data.fill_(0.0)
        self.lin2.bias.data.fill_(0.0)
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        torch.manual_seed(1)
        torch.cuda.manual_seed(1)
        x = self.dropout(x)
        x = self.lin1(x)
        x = nn.functional.elu(x)
        torch.manual_seed(1)
        torch.cuda.manual_seed(1)
        x = self.dropout(x)
        x = self.lin2(x)
        x = nn.functional.softmax(x, dim = 1)
        return x

In [18]:
import copy
from torchmetrics import Accuracy
import matplotlib.pyplot as plt

class Classifier():
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    metrics = dict({})
    metrics["train"] = []
    metrics["val"] = []
    metrics["test"] = []
    
    losses = dict({})
    losses["train"] = []
    losses["val"] = []
    losses["test"] = []
    
    models = []
    state_dicts = []
    
    best_state_dict = None
    best_metric = dict({})
    best_loss = dict({})
    best_model = None
    
    
    def __init__(self, X, y,lr=3e-2,weight_decay=1e-4, epochs=100_000, patience = 100, dropout=.6):
        num_classes = torch.unique(y["train"]).shape[0]
        assert num_classes == 7
        self.model = NeuralNet(X["train"].shape[1], 16, num_classes, dropout).to(Classifier.device)

        self.X = X
        self.y = y
        self.lr = lr
        self.weight_decay = weight_decay
        self.epochs = epochs
        self.patience = patience
        self.dropout = dropout
        self.optim = torch.optim.Adam(self.model.parameters(), lr=lr, weight_decay= weight_decay)
        self.metric_fn = Accuracy(task="multiclass", num_classes=num_classes).to(Classifier.device)
        self.loss_fn = nn.CrossEntropyLoss(reduction='mean')
        self.enough_training_for_today = False
        self.empty()

    def empty(self):
        for key in self.metrics:
            Classifier.metrics[key] = []
        for key in self.losses:
            Classifier.losses[key] = []
        Classifier.models = []
        Classifier.state_dicts = []
        
        Classifier.best_state_dict = None
        Classifier.best_metric = dict({})
        Classifier.best_loss = dict({})
        Classifier.best_model = None

    def fit(self, X, y):
        X = Classifier.dict_to_device(X)
        y = Classifier.dict_to_device(y)
        for epoch in range(self.epochs):
            self.model.train()
            logits = self.model(X["train"])
            self.optim.zero_grad()
            loss = self.loss_fn(logits, y["train"])
            loss.backward()
            self.optim.step()
            
            self.eval_all(X, y)
    
            if self.enough_training_for_today:
                self.store_best(self.patience)
                break
        self.store_best(0)

    def eval_all(self, X, y):
        self.evaluate(X["train"], y["train"], "train")
        self.evaluate(X["val"], y["val"], "val")
        self.evaluate(X["test"], y["test"], "test")

    def evaluate(self, x, y, set_name):
        with torch.inference_mode():
            self.model.eval()
            logits = self.model(x)
            loss = self.loss_fn(logits, y)
            metric = self.metric_fn(y, logits.argmax(1))
            if set_name == "val" and len(Classifier.losses[set_name]) >= self.patience:
                last_metrics = Classifier.metrics[set_name][-self.patience:]
                last_losses = Classifier.losses[set_name][-self.patience:]
                
                if all([(m >= metric.item()) for m in last_metrics]) or all([(l <= loss.item()) for l in last_losses]):
                    self.enough_training_for_today = True
    
            Classifier.metrics[set_name].append(metric.item())
            Classifier.losses[set_name].append(loss.item())
            Classifier.state_dicts.append(self.model.state_dict().copy())
            Classifier.models.append(copy.deepcopy(self.model))

    def store_best(self, offset):
        for key, value in Classifier.metrics.items():
                    Classifier.best_metric[key] = value[-1-offset]
        for key, value in Classifier.losses.items():
            Classifier.best_loss[key] = value[-1-offset]
        Classifier.best_state_dict = Classifier.state_dicts[-1-offset]
        Classifier.best_model = Classifier.models[-1-offset]

    @staticmethod
    def dict_to_device(store_dict):
        for key in store_dict:
            store_dict[key] = store_dict[key].to(Classifier.device)
        return store_dict

    @staticmethod
    def plot(store_dict, title):
        for key, value in store_dict.items():
            plt.plot(value)
        plt.legend(store_dict.keys())
        plt.title(title)
        plt.show()

    def get_params(self, **kwargs):
        return {'lr': self.lr, 'weight_decay': self.weight_decay, 'patience': self.patience, 'dropout': self.dropout, 'X': self.X, 'y':self.y}

    def set_params(self, **kwargs):
        self.lr = kwargs["lr"]
        self.weight_decay = kwargs["weight_decay"]
        self.patience = kwargs["patience"]
        self.dropout = kwargs["dropout"]
        # self.X = kwargs["X"]
        # self.y = kwargs["y"]

    def predict(self, X):
        with torch.inference_mode():
            self.model.eval()
            logits = self.model(x)
            return logits.argmax(1)

In [19]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops

dataset_name = 'Cora'
split = "public"
dataset = Planetoid(root='/tmp/Cora', name=dataset_name, split=split)
dataset.transform = T.NormalizeFeatures()

features =  dataset[0].x
labels =  dataset[0].y

test =  dataset[0].test_mask
train = dataset[0].train_mask
val =  dataset[0].val_mask

edge_index = dataset[0].edge_index 
edge_index = add_self_loops(edge_index)[0]

X = dict({})
X["train"] = features[train]
X["val"] = features[val]
X["test"] = features[test]

y = dict({})
y["train"] = labels[train]
y["val"] = labels[val]
y["test"] = labels[test]

In [20]:

def user_function(kwargs):
    return  kwargs["updated_features"] + kwargs["summed_neighbors"]

framework = Framework([user_function], 
                     hops_list=[1], ## to obtain best for local neighborhood
                     clfs=[],
                     gpu_idx=0,
                     handle_nan=0.0,
                    attention_configs=[{'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None}])
new_features = framework.get_features(features, edge_index, None)[0]
X = dict({})
X["train"] = new_features[train]
X["val"] = new_features[val]
X["test"] = new_features[test]

y = dict({})
y["train"] = labels[train]
y["val"] = labels[val]
y["test"] = labels[test]

In [21]:
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import ParameterGrid
from tqdm.notebook import tqdm

space = {
    'lr': np.linspace(1e-5, 1e-2, 30, endpoint=True),
    'weight_decay': np.linspace(1e-5, 1e-2, 30, endpoint=True),
    'patience': [20, 100],
    'dropout':[0, 0.1, 0.2,.3,.4,.5,.6]
}
param_grid = ParameterGrid(space)
best_score = 0
best_params = None
for params in tqdm(param_grid.__iter__()):
    model = Classifier(X, y,epochs=100_000, **params)
    model.fit(X, y)
    score = model.best_metric["val"]
    if score > best_score:
        best_score = score
        best_params = params
        print(best_score)
        # model.plot(model.losses, "Loss")

0it [00:00, ?it/s]

0.18199999630451202
0.20000000298023224
0.36000001430511475
0.3700000047683716
0.5320000052452087
0.5540000200271606
0.5640000104904175
0.5680000185966492
0.5740000009536743
0.5759999752044678
0.5799999833106995
0.5820000171661377
0.5839999914169312
0.5879999995231628
0.5920000076293945
0.593999981880188
0.5979999899864197
0.6019999980926514
0.6039999723434448
0.6060000061988831
0.6079999804496765


In [22]:
best_score
best_params


{'dropout': 0.3,
 'lr': 0.005866206896551724,
 'patience': 100,
 'weight_decay': 0.0072441379310344825}

In [23]:
model = Classifier(X, y,epochs=100_000, **best_params)
model.fit(X, y)
model.best_metric["val"]

0.6079999804496765